# this notebook generate g16 input file for 3-step TS pre-optimization at semi-empirical level

In [1]:
import os
import sys
import pandas as pd
import json

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [5]:
ts_data = list()
f = 'step2_initial_ts_guess.json'
with open(f, 'r') as openfile:
    ts_data.append(json.load(openfile))

In [6]:
ts_data.sort(key=lambda y:y['rxn_id'])

In [7]:
ts_data = tuple(ts_data)

In [8]:
g16_template = """%chk=rxn_{check}.chk
%mem={ramgb}gb
%NProcShared={cpu}

#P opt=(ModRedundant,calcall,maxcycle=128,noeig,nomicro) scf=(xqc) iop(7/33=1) iop(2/9=2000) iop(7/127=-99) iop(8/117=-99) am1

am1_opt

{charge} {multi}
{xyz}b {h_atom} {pivot_0} b
b {h_atom} {pivot_1} b
b {h_atom} {pivot_0} F
b {h_atom} {pivot_1} F
A {pivot_0} {h_atom} {pivot_1} F


--Link1--
%chk=rxn_{check}.chk
%mem={ramgb}gb
%NProcShared={cpu}

#P opt=(ModRedundant,ts,calcall,maxcycle=128,noeig,nomicro) scf=(xqc) iop(7/33=1) iop(2/9=2000) geom=check guess=mix pm7

pm7_opt_ts_freeze

{charge} {multi}

b {h_atom} {pivot_0} b
b {h_atom} {pivot_1} b
b {h_atom} {pivot_0} F
b {h_atom} {pivot_1} F
A {pivot_0} {h_atom} {pivot_1} F


--Link1--
%chk=rxn_{check}.chk
%mem={ramgb}gb
%NProcShared={cpu}

#P opt=(ts,calcall,maxcycle=256,noeig,nomicro,cartesian) scf=(xqc) iop(7/33=1) iop(2/9=2000) geom=check guess=mix
external="/home/gridsan/oscarwu/code/RDMC-main/rdmc/external/xtb_tools/xtb_gaussian.pl --gfn 2 -P"

xtb_opt_ts_free

{charge} {multi}





"""

In [9]:
for i, data in enumerate(ts_data):
       
    g16_save_dir = os.getcwd()
        
    if not os.path.exists(g16_save_dir):
        os.mkdir(g16_save_dir)
        
    smi = data['rxn_smi']
    rxn_id = data['rxn_id']
    ts_gemo_info = data['ts_geom_info']
    h_atom = data['ts_geom_info'][0] + 1
    pivot_0 = data['ts_geom_info'][1] + 1
    pivot_1 = data['ts_geom_info'][2] + 1
    xyz = data['conformers'][0][1]
    
    g16_input_path = os.path.join(g16_save_dir, f'rxn_{rxn_id}_ts_xtb_opt.gjf')
    
    with open(g16_input_path, 'w') as f:
        f.write(g16_template.format(check=rxn_id, 
                                    ramgb=46,
                                    cpu=46,
                                    charge=0,
                                    multi=2,
                                    xyz=xyz,
                                    h_atom=h_atom,
                                    pivot_0=pivot_0,
                                    pivot_1=pivot_1,
        ))